# Búsquedas facetadas

En este notebook vamos a explorar las diferentes capacidades de búsqueda que nos ofrece Elasticserach. En concreto vamos a ver como hacer:

<img src="../images/els/els31.jpg" alt="Busqueda autocompletada"/>

Para realizar los ejercicios propuestos en este notebook, puedes utilizar el Dev Tools de Kibana. Puedes acceder a él a través de este enlace: http://127.0.0.1:5601

Como paso previo vamos a crear el índice y a insertar unos cuantos datos en él para poder realizar los ejercicios.

Vamos a utilizar la sentencia bulk como hicimos en el notebook anterior.

`POST /bookdb_index/book/_bulk`

# Búsquedas facetadas

Permite buscar un documento filtrando por una o varias de sus propiedades (facetas).

Para construir filtros facetados se utiliza el framework de agregación y más concretamente las agregaciones de tipo Bucket Aggregation. 

El lenguaje de consulta permite añadir un contexto de agregación junto al contexto de búsqueda y filtrado. Con el resultado de la búsqueda se adjunta el resultado de la agregación.

La agregación siempre se aplica al subconjunto de documentos que cumplen los criterios de búsqueda definidos en el contexto de búsqueda y filtrado.

Las agregaciones de tipo bucket generan como resultado buckets, donde cada bucket está asociado con una clave y a un criterio de selección. 

Cuando se ejecuta la agregación, todos los criterios asociados a cada bucket se evalúan sobre cada documento que encontramos en el contexto y si estos criterios se cumplen se incluye en el bucket. 

Al final de la ejecución se obtiene una lista de los buckets con el conjunto de documentos que pertenece a cada uno de ellos.

Dependiendo del tipo del campo sobre el que vamos ha hacer las facetas utilizaremos las siguientes agregaciones (en este notebook sólo vamos a ver las más comunes. Para ver el total de agregaciones que se pueden realizar en Elasticsearch consultar la siguiente página de la documentación: https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket.html):

* Texto:
    * Terms Aggregation: los buckets se generan de forma dinámica para cada uno de los valores únicos que se encuentran en los campos especificados en la agregación.
* Números:
    * Range Aggregation
    * Histogram Aggregation
* Fechas:
    * Date Range Aggregation
    * Date Histogram Aggregation


## 1. Terms Aggregation

Utilizaremos terms aggregation cuando queremos extraer facetas de campos de tipo texto. En concreto, la agregación Terms Aggregation sólo funciona sobre campos de tipo **keyword**. Si el campo a nalalizar es de tipo text, podemos utilizar los fielddata.

Si lo que quermos es crear una búsqueda por la faceta género, realizaremos una agregación por el campo "genere". Ejecuta la siguiente sentencia en Kibana y vamos a analizar el resultado.

`GET /_search
{
    "aggs" : {
        "genres" : {
            "terms" : { "field" : "genre" } 
        }
    }
}
`
En la información devuelta vamos a fijarnos en el campo "aggregations". ¿Podrídas contestar a las siguietnes preguntas? 

1. Para la faceta "generes" ¿cuántos buckets diferentes hay?
2. Cuantos documentos ha encontrado que contenga el genero rock en la faceta género?
3. ¿Hay algún orden en el que devuelva los bickets?

Por defecto Elasticsearch devuelve los 10 buckets con más elementos. Si queremos cambiar este comportamiento, podemos utilizar el parámetro size:

`GET /_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "size" : 5
            }
        }
    }
}`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

Elasticsearch también nos permite cambiar el orden en el que devuelve los buckets. Por ejemplo, si queremos devolver los bockes ordenados de forma descendente por el número de documentos que contiene:

`GET /_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "order" : { "_count" : "asc" }
            }
        }
    }
}`

1. ¿Cómo ordena los buckts esta sentencia?

Pero además también podemos ordenar el resultado por el valor de la clave del bucket, por ejemplo, si queremos ordenar los resultados por orden alfabético:

`GET /_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "order" : { "_key" : "asc" }
            }
        }
    }
}`

También podemos especificar el número mínimo de documentos que tiene que haber en cada bucket para que se incluya en el resultado.

`GET /_search
{
    "aggs" : {
        "genre" : {
            "terms" : {
                "field" : "genre",
                "min_doc_count": 10
            }
        }
    }
}`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

### 1.1 Ejercicio: Realiza una búsqueda por dos facetas, género y discográfica.
### 1.2 Ejercicio: Excluye del resultado los buckets que no tengan más de 10 documentos.
### 1.3 Ejercicio: Ordena el resultado de los buckets por el orden alfabéico del nombre de la discográfica.

## 2. Range Aggregation

Se utilizan sobre campos numéricos y permiten definir el rango de valores que puede tener cada bucket. Para ello se especifica el valor máximo y mínimo entre los que tien que estar el valor de una faceta para que se incluya en ese bucket.

`GET /_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "ranges" : [
                    { "to" : 100.0 },
                    { "from" : 100.0, "to" : 200.0 },
                    { "from" : 200.0 }
                ]
            }
        }
    }
}`

1. ¿Cuantos buckets nos devuelve esta consulta?

Podemos decirle a Elasticsearch que incluya como nombre del bucket el rango definido utilizando el parámetro "keyed" en la query:

`GET /_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "keyed" : true,
                "ranges" : [
                    { "to" : 100 },
                    { "from" : 100, "to" : 200 },
                    { "from" : 200 }
                ]
            }
        }
    }
}`

1. ¿Qué nombre tienen ahora los buckets?

Si queremos darle la misma semántica a los nombres que en el dominio que estamos modelando, podemos definir un nombre a cada bucket:

`GET /_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "keyed" : true,
                "ranges" : [
                    { "key" : "cheap", "to" : 100 },
                    { "key" : "average", "from" : 100, "to" : 200 },
                    { "key" : "expensive", "from" : 200 }
                ]
            }
        }
    }
}`

### 2.1 Ejercicio: Realiza una búsqueda que devuleva el número de discos que hay función del número de tracks que tiene cada uno, SinglePlay de hasta 3, StandarPlay: de 3 a 10 y LongPlay: mas de 10. 
### 2.2 Ejercicio: Pon como nombre del buket SingelP, SandarP y LongP  según corresponda.

## 3. Date Range Aggregation

Funciona igual que la agregación Range que hemos visto antes, pero con campos de tipo fecha. Además permite expresiones específicas para fechas para definir los rangos.

Entra en este enlace para ver las expresiones matemáticas que se pueden utilizar: https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#date-math

`POST /sales/_search?size=0
{
    "aggs": {
        "range": {
            "date_range": {
                "field": "date",
                "format": "MM-yyyy",
                "ranges": [
                    { "to": "now-10M/M" }, 
                    { "from": "now-10M/M" } 
                ]
            }
        }
    }
}`

### 3.1 Ejercicio: Realiza una búsqueda que devuelva el número de discos que hay por década desde 1900 hasta la actualidad.
### 3.2 Ejercicio: Pon como nobre del bucket la década a la que pertenence el resultado.


## 4. Histogram Aggregation

Se tuiliza con campos de tipo numérico. Permite realizar buckets en función de la sitribución de los datos de un campo. Para ello es necesario definir el número fijo de valores que puede haber entre el valor máximo y mínimo del bucket.

Por ejemplo si tenemos en cuenta el campo precio de nuestro índice, si le indicamos que cree los buckets con un intervalo de 5 (5€), cuando evalúe el valor del campo precio para cada documento lo insertará en el bucket mas próximo por redondeo. Por ejemplo, para le valor 30 lo insertará en el bucket 30.

` 
POST /sales/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 50
            }
        }
    }
}`

1. ¿Cómo se incrementan los valores de los buckets según el resultado?

Como pasara con el Terms Aggregation, se puede definir el número mínimo de documentos que tiene que tener el bucket para que aparezca en el resultado.

`POST /sales/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 50,
                "min_doc_count" : 1
            }
        }
    }
}`

También permiten que el nombre del bucket sea el rango del intervalo.

`POST /sales/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 50,
                "keyed" : true
            }
        }
    }
}`

### 4.1 Ejercicio: Realiza una búsqueda que devuelva el número de discos que hay por rating con un intervalo de 10.

## 5. Date Histogram Aggregation

Es igual que el Histogram Aggregation, pero con campos de tipo fecha. En este caso los inervalos pueden ser:

* minute (m, 1m)
* hour (h, 1h)
* day (d, 1d)
* week (w, 1w)
* month (M, 1M)
* quarter (q, 1q)
* year (y, 1y)

`POST /sales/_search?size=0
{
    "aggs" : {
        "sales_over_time" : {
            "date_histogram" : {
                "field" : "date",
                "calendar_interval" : "2d"
            }
        }
    }
}`

### 5.1 Ejercicio: Realiza el ejercicio 3.1 pero con Date Histogram 